<a href="https://colab.research.google.com/github/aSafarpoor/OSN_FAD/blob/main/Data/Twibot20_test_with_simple_GCN_and_GAT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 22.3 MB/s eta 0:00:00


In [17]:
import networkx as nx
import numpy as np
from tqdm import tqdm
from matplotlib import pyplot as plt
import random


import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, GATConv, GINConv
from torch.nn import Module, Linear
from sklearn.metrics import roc_curve,accuracy_score, roc_auc_score, confusion_matrix
import torch.nn as nn


SEED = 10
random.seed(SEED)

#read files

In [3]:
import tarfile

file_name = "Twibot20.tar.xz"

with tarfile.open(file_name, "r:xz") as tar:
    tar.extractall()
    tar.list()

?r-xr-xr-x 0/0          0 2024-11-19 03:33:21 Twibot20/ 
?rw-rw-rw- 0/0      67324 2024-11-19 03:33:06 Twibot20/benigns.txt 
?rw-rw-rw- 0/0    6649660 2024-11-18 00:45:41 Twibot20/edges.txt 
?rw-rw-rw- 0/0      95339 2024-11-19 03:33:11 Twibot20/sybils.txt 
?rw-rw-rw- 0/0    2410572 2024-11-18 09:16:36 Twibot20/testNodes.txt 
?rw-rw-rw- 0/0     522736 2024-11-18 09:16:41 Twibot20/trainNodes.txt 


In [4]:
file_name = "Twibot20/benigns.txt"
with open(file_name, "r") as file:
    lines = file.readlines()
benigns = [line.strip() for line in lines]

file_name = "Twibot20/sybils.txt"
with open(file_name, "r") as file:
    lines = file.readlines()
sybils = [line.strip() for line in lines]

file_name = "Twibot20/trainNodes.txt"
with open(file_name, "r") as file:
    lines = file.readlines()
trainNodes = [line.strip() for line in lines]

file_name = "Twibot20/testNodes.txt"
with open(file_name, "r") as file:
    lines = file.readlines()
testNodes = [line.strip() for line in lines]

edges = []
file_name = "Twibot20/edges.txt"

with open(file_name, "r") as file:
    edges = [tuple( line.strip().split()) for line in file]

In [5]:
def renumber_nodes(trainNodes, testNodes, benigns, sybils, edges):
    train_mapping = {node: idx for idx, node in enumerate(trainNodes)}
    test_start_idx = len(trainNodes)
    test_mapping = {node: idx for idx, node in enumerate(testNodes, start=test_start_idx)}
    node_mapping = {**train_mapping, **test_mapping}
    updated_trainNodes = [node_mapping[node] for node in trainNodes]
    updated_testNodes = [node_mapping[node] for node in testNodes]
    updated_benigns = [node_mapping[node] for node in benigns if node in node_mapping]
    updated_sybils = [node_mapping[node] for node in sybils if node in node_mapping]
    updated_edges = [(node_mapping[src], node_mapping[dst]) for src, dst in edges if src in node_mapping and dst in node_mapping]
    return updated_trainNodes, updated_testNodes, updated_benigns, updated_sybils, updated_edges

trainNodes, testNodes, benigns, sybils, edges = renumber_nodes(trainNodes, testNodes, benigns, sybils, edges)

# Models

In [6]:
class GCN(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers, dropout=True):
        super(GCN, self).__init__()
        self.dropout = dropout
        self.convs = torch.nn.ModuleList()
        self.num_classes = output_dim
        for i in range(num_layers):
            if i == 0:  # First layer
                self.convs.append(GCNConv(input_dim, hidden_dim, bias=True))
            elif i == num_layers - 1:  # Last layer
                self.convs.append(GCNConv(hidden_dim, output_dim, bias=True))
            else:  # Middle layers
                self.convs.append(GCNConv(hidden_dim, hidden_dim, bias=True))

    def forward(self, x, edge_index):
        h = x
        i = 0
        for conv in self.convs:
            if self.dropout:
                h = F.dropout(h, p=0.5, training=self.training)
            h = conv(h, edge_index)
            if i < len(self.convs) - 1:
                h = F.tanh(h)
            i += 1

        if self.num_classes == 1:
            return F.sigmoid(h)
        else:
            return F.softmax(h, dim=1)

In [7]:
class GAT(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers, heads=1, dropout=True):
        super(GAT, self).__init__()
        self.dropout = dropout
        self.convs = torch.nn.ModuleList()
        self.num_classes = output_dim
        self.heads = heads

        for i in range(num_layers):
            if i == 0:  # First layer
                self.convs.append(GATConv(input_dim, hidden_dim, heads=heads, concat=True, bias=True))
            elif i == num_layers - 1:  # Last layer
                self.convs.append(GATConv(hidden_dim * heads, output_dim, heads=1, concat=False, bias=True))
            else:  # Middle layers
                self.convs.append(GATConv(hidden_dim * heads, hidden_dim, heads=heads, concat=True, bias=True))

    def forward(self, x, edge_index):
        h = x
        for i, conv in enumerate(self.convs):
            if self.dropout:
                h = F.dropout(h, p=0.5, training=self.training)
            h = conv(h, edge_index)
            if i < len(self.convs) - 1:  # Apply activation only for hidden layers
                h = F.elu(h)  # ELU is often preferred for GAT
        if self.num_classes == 1:
            return F.sigmoid(h)
        else:
            return F.log_softmax(h, dim=1)

In [23]:
from torch_geometric.nn import APPNP

class APPNPModel(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers, dropout=True, alpha=0.1, K=10):
        """
        - alpha: Teleport probability (1 - alpha is the damping factor).
        - k: Number of propagation steps.
        """
        super(APPNPModel, self).__init__()
        self.dropout = dropout
        self.num_classes = output_dim

        # Define MLP layers
        self.layers = torch.nn.ModuleList()
        for i in range(num_layers):
            if i == 0:  # First layer
                self.layers.append(Linear(input_dim, hidden_dim))
            elif i == num_layers - 1:  # Last layer
                self.layers.append(Linear(hidden_dim, output_dim))
            else:  # Middle layers
                self.layers.append(Linear(hidden_dim, hidden_dim))

        # APPNP propagation
        self.prop = APPNP(K=K, alpha=alpha)

    def forward(self, x, edge_index):
        h = x
        # Pass through MLP
        for i, layer in enumerate(self.layers):
            if self.dropout:
                h = F.dropout(h, p=0.5, training=self.training)
            h = layer(h)
            if i < len(self.layers) - 1:  # Apply activation and dropout only for hidden layers
                h = F.relu(h)

        # Apply APPNP propagation
        h = self.prop(h, edge_index)

        if self.num_classes == 1:
            return F.sigmoid(h)
        else:
            return F.log_softmax(h, dim=1)

In [8]:
def create_datasets(all_nodes, train_nodes, test_nodes, benign_nodes, sybil_nodes, ratio_of_known_trains, ratio_of_known_tests,SEED):

    random.seed(SEED)

    labels = {node: 0.5 for node in all_nodes}  # Unknown nodes default to 0.5
    for node in benign_nodes:
        labels[node] = 0
    for node in sybil_nodes:
        labels[node] = 1

    train_x = [labels[node] for node in train_nodes]
    for i in range(len(train_x)):
        if train_x[i] != 0.5:
            if random.random()>ratio_of_known_trains:
                train_x[i] = 0.5
    train_y = [labels[node] for node in train_nodes]


    test_x = [labels[node] for node in test_nodes]
    for i in range(len(test_x)):
        if test_x[i] != 0.5:
            if random.random()>ratio_of_known_tests:
                test_x[i] = 0.5
    test_y = [labels[node] for node in test_nodes]

    mask_test = [i != 0.5 for i in test_y]
    mask_test_sybils = [i == 1 for i in test_y]


    train_x = torch.tensor(train_x, dtype=torch.float).reshape(-1,1)
    train_y = torch.tensor(train_y, dtype=torch.float).reshape(-1,1)
    test_x = torch.tensor(test_x, dtype=torch.float).reshape(-1,1)
    test_y = torch.tensor(test_y, dtype=torch.float).reshape(-1,1)

    return train_x, test_x, train_y, test_y, mask_test, mask_test_sybils

In [9]:
train_x, test_x, train_y, test_y, mask_test, mask_test_sybils = create_datasets(all_nodes = trainNodes + testNodes,
                                                                                train_nodes = trainNodes,
                                                                                test_nodes = testNodes,
                                                                                benign_nodes = benigns,
                                                                                sybil_nodes = sybils,
                                                                                ratio_of_known_trains = 0.5,
                                                                                ratio_of_known_tests = 0.5,
                                                                                SEED = 1)

In [10]:
g = nx.Graph()
g.add_edges_from(edges)


temp = list(g.subgraph(trainNodes).edges())
trainEdges = [[i[0] for i in temp],[i[1] for i in temp[:]]]
temp = list(g.subgraph(testNodes).edges())
testEdges = [[i[0] for i in temp],[i[1] for i in temp[:]]]

trainEdges = torch.tensor(trainEdges, dtype=torch.long)
testEdges = torch.tensor(testEdges, dtype=torch.long)


In [11]:
def train_model(model, train_x, train_y, train_edge_index, epochs=100, lr=0.01):

    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    # loss_fn = torch.nn.CrossEntropyLoss()
    loss_fn = torch.nn.MSELoss()


    model.train()
    for epoch in tqdm(range(epochs)):
        optimizer.zero_grad()
        output = model(train_x, train_edge_index)
        loss = loss_fn(output, train_y)
        loss.backward()
        optimizer.step()

        if (epoch + 1) % 20 == 0 or epoch == 0:
            print(f'Epoch {epoch + 1}/{epochs}, Loss: {loss.item():.4f}')

    return model



In [12]:
def test_model(model, test_x, test_y, test_edge_index, mask=None, threshold=None):
    model.eval()
    with torch.no_grad():
        output = model(test_x, test_edge_index)

        # Apply mask if provided
        if mask is not None:
            output = output[mask]
            test_y = test_y[mask]

        # If output has shape [batch_size, 1], use the single column
        output = output.squeeze(1)  # Converts shape [batch_size, 1] -> [batch_size]

        # Compute AUC and find optimal threshold
        try:
            auc_score = roc_auc_score(test_y.cpu(), output.cpu())  # Adjust for single-class output
            # Compute ROC curve
            fpr, tpr, thresholds = roc_curve(test_y.cpu(), output.cpu())
            # Find optimal threshold using Youden's J statistic
            j_scores = tpr - fpr
            optimal_idx = j_scores.argmax()
            if threshold is None:
                threshold = thresholds[optimal_idx]

        except Exception as e:
            auc_score = None
            if threshold is None:
                threshold = 0.5  # Default threshold if AUC computation fails

        # Apply the threshold to classify predictions
        threshold_predictions = (output >= threshold).int().cpu()

        # Compute accuracy using the threshold
        accuracy_with_threshold = accuracy_score(test_y.cpu(), threshold_predictions)

        # Compute confusion matrix
        conf_matrix = confusion_matrix(test_y.cpu(), threshold_predictions)
        normalized_cm = conf_matrix.astype('float') / len(test_y)

        # Prepare metrics
        metrics = {
            "accuracy": accuracy_with_threshold,
            "auc": auc_score,
            "optimal_threshold": threshold,
            "confusion_matrix": normalized_cm
        }

        # Print metrics
        try:
            print(f"AUC: {auc_score:.3f}")
        except:
            pass
        # print(f"Optimal Threshold: {threshold}")
        print(f"ACC: {accuracy_with_threshold:.3f}")
        print("Confusion Matrix (Normalized):")
        print(f"                      Predicted Negative                       Predicted Positive")
        print(f"Actual Negative       {normalized_cm[0, 0]:.3f}                {normalized_cm[0, 1]:.3f}")
        print(f"Actual Positive       {normalized_cm[1, 0]:.3f}                {normalized_cm[1, 1]:.3f}")

        return metrics, threshold


In [13]:
input_dim = 1
hidden_dim = 16
output_dim = 1
model = GCN(input_dim, hidden_dim, output_dim, num_layers=3)

model = train_model(model, train_x, train_y, trainEdges, epochs=100, lr=0.01)



print("based on auc th")
metric,th = test_model(model, test_x, test_y, testEdges-len(train_x), mask = mask_test)
metric,th = test_model(model, test_x, test_y, testEdges-len(train_x), mask = mask_test_sybils, threshold = th)
print("\n\n")
print("based on 0.5")
metric,th = test_model(model, test_x, test_y, testEdges-len(train_x), mask = mask_test, threshold = 0.5)
metric,th = test_model(model, test_x, test_y, testEdges-len(train_x), mask = mask_test_sybils, threshold = 0.5)

  1%|          | 1/100 [00:00<00:50,  1.95it/s]

Epoch 1/100, Loss: 0.0245


 21%|██        | 21/100 [00:03<00:13,  5.70it/s]

Epoch 20/100, Loss: 0.0174


 40%|████      | 40/100 [00:09<00:19,  3.11it/s]

...

 81%|████████  | 81/100 [00:17<00:03,  5.37it/s]

Epoch 80/100, Loss: 0.0169


100%|██████████| 100/100 [00:20<00:00,  4.83it/s]

Epoch 100/100, Loss: 0.0169
based on auc th


AUC: 0.661
ACC: 0.616
Confusion Matrix (Normalized):
                      Predicted Negative                       Predicted Positive
Actual Negative       0.274                0.145
Actual Positive       0.239                0.342
ACC: 0.589
Confusion Matrix (Normalized):
                      Predicted Negative                       Predicted Positive
Actual Negative       0.000                0.000
Actual Positive       0.411                0.589



based on 0.5
AUC: 0.661
ACC: 0.550
Confusion Matrix (Normalized):
                      Predicted Negative                       Predicted Positive
Actual Negative       0.362                0.056
Actual Positive       0.394                0.188
ACC: 0.323
Confusion Matrix (Normalized):
                      Predicted Negative                       Predicted Positive
Actual Negative       0.000                0.000
Actual Positive       0.677                0.323


In [14]:
input_dim = 1
hidden_dim = 16
output_dim = 1
model = GAT(input_dim, hidden_dim, output_dim, num_layers=3, heads = 4)

model = train_model(model, train_x, train_y, trainEdges, epochs=100, lr=0.01)


print("based on auc th")

metric,th = test_model(model, test_x, test_y, testEdges-len(train_x), mask = mask_test)
metric,th = test_model(model, test_x, test_y, testEdges-len(train_x), mask = mask_test_sybils, threshold = th)
print("\n\n")
print("based on 0.5")
metric,th = test_model(model, test_x, test_y, testEdges-len(train_x), mask = mask_test, threshold = 0.5)
metric,th = test_model(model, test_x, test_y, testEdges-len(train_x), mask = mask_test_sybils, threshold = 0.5)

  1%|          | 1/100 [00:00<00:53,  1.84it/s]

Epoch 1/100, Loss: 0.0198


 20%|██        | 20/100 [00:08<00:22,  3.52it/s]

Epoch 20/100, Loss: 0.0177


 40%|████      | 40/100 [00:14<00:20,  2.94it/s]

...

 80%|████████  | 80/100 [00:27<00:06,  3.19it/s]

Epoch 80/100, Loss: 0.0169


100%|██████████| 100/100 [00:33<00:00,  3.00it/s]

Epoch 100/100, Loss: 0.0169
based on auc th


AUC: 0.744
ACC: 0.658
Confusion Matrix (Normalized):
                      Predicted Negative                       Predicted Positive
Actual Negative       0.321                0.097
Actual Positive       0.246                0.336
ACC: 0.578
Confusion Matrix (Normalized):
                      Predicted Negative                       Predicted Positive
Actual Negative       0.000                0.000
Actual Positive       0.422                0.578



based on 0.5
AUC: 0.744
ACC: 0.510
Confusion Matrix (Normalized):
                      Predicted Negative                       Predicted Positive
Actual Negative       0.410                0.009
Actual Positive       0.481                0.100
ACC: 0.173
Confusion Matrix (Normalized):
                      Predicted Negative                       Predicted Positive
Actual Negative       0.000                0.000
Actual Positive       0.827                0.173


In [15]:
input_dim = 1
hidden_dim = 8
output_dim = 1
model = GAT(input_dim, hidden_dim, output_dim, num_layers=4, heads = 4)

model = train_model(model, train_x, train_y, trainEdges, epochs=100, lr=0.01)


print("based on auc th")

metric,th = test_model(model, test_x, test_y, testEdges-len(train_x), mask = mask_test)
metric,th = test_model(model, test_x, test_y, testEdges-len(train_x), mask = mask_test_sybils, threshold = th)
print("\n\n")
print("based on 0.5")
metric,th = test_model(model, test_x, test_y, testEdges-len(train_x), mask = mask_test, threshold = 0.5)
metric,th = test_model(model, test_x, test_y, testEdges-len(train_x), mask = mask_test_sybils, threshold = 0.5)

  1%|          | 1/100 [00:00<00:41,  2.40it/s]

Epoch 1/100, Loss: 0.0326


 20%|██        | 20/100 [00:06<00:22,  3.61it/s]

Epoch 20/100, Loss: 0.0173


 40%|████      | 40/100 [00:14<00:26,  2.29it/s]

...

 80%|████████  | 80/100 [00:28<00:07,  2.76it/s]

Epoch 80/100, Loss: 0.0169


100%|██████████| 100/100 [00:34<00:00,  2.90it/s]

Epoch 100/100, Loss: 0.0169
based on auc th


AUC: 0.721
ACC: 0.647
Confusion Matrix (Normalized):
                      Predicted Negative                       Predicted Positive
Actual Negative       0.308                0.110
Actual Positive       0.243                0.339
ACC: 0.582
Confusion Matrix (Normalized):
                      Predicted Negative                       Predicted Positive
Actual Negative       0.000                0.000
Actual Positive       0.418                0.582



based on 0.5
AUC: 0.721
ACC: 0.418
Confusion Matrix (Normalized):
                      Predicted Negative                       Predicted Positive
Actual Negative       0.418                0.000
Actual Positive       0.582                0.000
ACC: 0.000
Confusion Matrix (Normalized):
                      Predicted Negative                       Predicted Positive
Actual Negative       0.000                0.000
Actual Positive       1.000                0.000


In [16]:
input_dim = 1
hidden_dim = 8
output_dim = 1
model = GAT(input_dim, hidden_dim, output_dim, num_layers=4, heads = 4)

model = train_model(model, train_x, train_y, trainEdges, epochs=500, lr=0.01)


print("based on auc th")

metric,th = test_model(model, test_x, test_y, testEdges-len(train_x), mask = mask_test)
metric,th = test_model(model, test_x, test_y, testEdges-len(train_x), mask = mask_test_sybils, threshold = th)
print("\n\n")
print("based on 0.5")
metric,th = test_model(model, test_x, test_y, testEdges-len(train_x), mask = mask_test, threshold = 0.5)
metric,th = test_model(model, test_x, test_y, testEdges-len(train_x), mask = mask_test_sybils, threshold = 0.5)

  0%|          | 1/500 [00:00<04:09,  2.00it/s]

Epoch 1/500, Loss: 0.0228


  4%|▍         | 20/500 [00:10<02:18,  3.47it/s]

Epoch 20/500, Loss: 0.0170


  8%|▊         | 40/500 [00:15<02:12,  3.48it/s]

...

 96%|█████████▌| 480/500 [02:44<00:05,  3.76it/s]

Epoch 480/500, Loss: 0.0169


100%|██████████| 500/500 [02:49<00:00,  2.95it/s]

Epoch 500/500, Loss: 0.0169
based on auc th


AUC: 0.720
ACC: 0.646
Confusion Matrix (Normalized):
                      Predicted Negative                       Predicted Positive
Actual Negative       0.308                0.110
Actual Positive       0.244                0.338
ACC: 0.581
Confusion Matrix (Normalized):
                      Predicted Negative                       Predicted Positive
Actual Negative       0.000                0.000
Actual Positive       0.419                0.581



based on 0.5
AUC: 0.720
ACC: 0.494
Confusion Matrix (Normalized):
                      Predicted Negative                       Predicted Positive
Actual Negative       0.413                0.006
Actual Positive       0.501                0.081
ACC: 0.139
Confusion Matrix (Normalized):
                      Predicted Negative                       Predicted Positive
Actual Negative       0.000                0.000
Actual Positive       0.861                0.139


In [26]:
input_dim = 1
hidden_dim = 16
output_dim = 1
model = APPNPModel(input_dim, hidden_dim, output_dim, num_layers=3, dropout=True, alpha=0.1, K=10)

model = train_model(model, train_x, train_y, trainEdges, epochs=100, lr=0.01)


print("based on auc th")

metric,th = test_model(model, test_x, test_y, testEdges-len(train_x), mask = mask_test)
metric,th = test_model(model, test_x, test_y, testEdges-len(train_x), mask = mask_test_sybils, threshold = th)
print("\n\n")
print("based on 0.5")
metric,th = test_model(model, test_x, test_y, testEdges-len(train_x), mask = mask_test, threshold = 0.5)
metric,th = test_model(model, test_x, test_y, testEdges-len(train_x), mask = mask_test_sybils, threshold = 0.5)

  3%|▎         | 3/100 [00:00<00:03, 24.35it/s]

Epoch 1/100, Loss: 0.0256


 24%|██▍       | 24/100 [00:01<00:03, 21.57it/s]

Epoch 20/100, Loss: 0.0172


 42%|████▏     | 42/100 [00:01<00:02, 21.54it/s]

...

Epoch 60/100, Loss: 0.0169


 84%|████████▍ | 84/100 [00:03<00:00, 21.92it/s]

Epoch 80/100, Loss: 0.0169


100%|██████████| 100/100 [00:04<00:00, 22.07it/s]


Epoch 100/100, Loss: 0.0169
based on auc th
AUC: 0.620
ACC: 0.602
Confusion Matrix (Normalized):
                      Predicted Negative                       Predicted Positive
Actual Negative       0.294                0.125
Actual Positive       0.273                0.308
ACC: 0.530
Confusion Matrix (Normalized):
                      Predicted Negative                       Predicted Positive
Actual Negative       0.000                0.000
Actual Positive       0.470                0.530



based on 0.5
AUC: 0.620
ACC: 0.418
Confusion Matrix (Normalized):
                      Predicted Negative                       Predicted Positive
Actual Negative       0.418                0.000
Actual Positive       0.582                0.000
ACC: 0.000
Confusion Matrix (Normalized):
                      Predicted Negative                       Predicted Positive
Actual Negative       0.000                0.000
Actual Positive       1.000                0.000


In [39]:
input_dim = 1
hidden_dim = 32
output_dim = 1
model = APPNPModel(input_dim, hidden_dim, output_dim, num_layers=4, dropout=True, alpha=0.1, K=10)


model = train_model(model, train_x, train_y, trainEdges, epochs=500, lr=0.01)


print("based on auc th")

metric,th = test_model(model, test_x, test_y, testEdges-len(train_x), mask = mask_test)
metric,th = test_model(model, test_x, test_y, testEdges-len(train_x), mask = mask_test_sybils, threshold = th)
print("\n\n")
print("based on 0.5")
metric,th = test_model(model, test_x, test_y, testEdges-len(train_x), mask = mask_test, threshold = 0.5)
metric,th = test_model(model, test_x, test_y, testEdges-len(train_x), mask = mask_test_sybils, threshold = 0.5)

  0%|          | 2/500 [00:00<00:35, 13.84it/s]

Epoch 1/500, Loss: 0.0189


  4%|▍         | 22/500 [00:01<00:35, 13.33it/s]

Epoch 20/500, Loss: 0.0170


  8%|▊         | 42/500 [00:03<00:33, 13.62it/s]

...

Epoch 460/500, Loss: 0.0169


 96%|█████████▋| 482/500 [00:39<00:01, 13.41it/s]

Epoch 480/500, Loss: 0.0169


100%|██████████| 500/500 [00:40<00:00, 12.22it/s]


Epoch 500/500, Loss: 0.0169
based on auc th
AUC: 0.634
ACC: 0.619
Confusion Matrix (Normalized):
                      Predicted Negative                       Predicted Positive
Actual Negative       0.271                0.147
Actual Positive       0.234                0.348
ACC: 0.598
Confusion Matrix (Normalized):
                      Predicted Negative                       Predicted Positive
Actual Negative       0.000                0.000
Actual Positive       0.402                0.598



based on 0.5
AUC: 0.634
ACC: 0.418
Confusion Matrix (Normalized):
                      Predicted Negative                       Predicted Positive
Actual Negative       0.418                0.000
Actual Positive       0.582                0.000
ACC: 0.000
Confusion Matrix (Normalized):
                      Predicted Negative                       Predicted Positive
Actual Negative       0.000                0.000
Actual Positive       1.000                0.000


In [29]:

from torch_geometric.nn import SSGConv

class SSGConvModel(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers, dropout=True, alpha=0.1, K=10):
        """
        SSGConv Model.
        - alpha: Teleport probability (controls the mixing of features).
        - K: Number of propagation steps.
        """
        super(SSGConvModel, self).__init__()
        self.dropout = dropout
        self.num_classes = output_dim

        # Define SSGConv layers
        self.layers = torch.nn.ModuleList()
        for i in range(num_layers):
            if i == 0:  # First layer
                self.layers.append(SSGConv(input_dim, hidden_dim, K=K, alpha=alpha))
            elif i == num_layers - 1:  # Last layer
                self.layers.append(SSGConv(hidden_dim, output_dim, K=K, alpha=alpha))
            else:  # Middle layers
                self.layers.append(SSGConv(hidden_dim, hidden_dim, K=K, alpha=alpha))

    def forward(self, x, edge_index):
        h = x
        for i, layer in enumerate(self.layers):
            if self.dropout:
                h = F.dropout(h, p=0.5, training=self.training)
            h = layer(h, edge_index)
            if i < len(self.layers) - 1:  # Apply activation only for hidden layers
                h = F.relu(h)

        if self.num_classes == 1:
            return F.sigmoid(h)
        else:
            return F.log_softmax(h, dim=1)


In [31]:
input_dim = 1
hidden_dim = 8
output_dim = 1
model = SSGConvModel(input_dim, hidden_dim, output_dim, num_layers=5, dropout=True, alpha=0.1, K=10)


model = train_model(model, train_x, train_y, trainEdges, epochs=100, lr=0.01)


print("based on auc th")

metric,th = test_model(model, test_x, test_y, testEdges-len(train_x), mask = mask_test)
metric,th = test_model(model, test_x, test_y, testEdges-len(train_x), mask = mask_test_sybils, threshold = th)
print("\n\n")
print("based on 0.5")
metric,th = test_model(model, test_x, test_y, testEdges-len(train_x), mask = mask_test, threshold = 0.5)
metric,th = test_model(model, test_x, test_y, testEdges-len(train_x), mask = mask_test_sybils, threshold = 0.5)

  1%|          | 1/100 [00:00<01:07,  1.48it/s]

Epoch 1/100, Loss: 0.0268


 20%|██        | 20/100 [00:10<00:44,  1.79it/s]

Epoch 20/100, Loss: 0.0172


 40%|████      | 40/100 [00:19<00:31,  1.89it/s]

...

 80%|████████  | 80/100 [00:38<00:09,  2.08it/s]

Epoch 80/100, Loss: 0.0169


100%|██████████| 100/100 [00:46<00:00,  2.14it/s]

Epoch 100/100, Loss: 0.0169
based on auc th


AUC: 0.590
ACC: 0.561
Confusion Matrix (Normalized):
                      Predicted Negative                       Predicted Positive
Actual Negative       0.312                0.106
Actual Positive       0.333                0.249
ACC: 0.428
Confusion Matrix (Normalized):
                      Predicted Negative                       Predicted Positive
Actual Negative       0.000                0.000
Actual Positive       0.572                0.428



based on 0.5
AUC: 0.590
ACC: 0.423
Confusion Matrix (Normalized):
                      Predicted Negative                       Predicted Positive
Actual Negative       0.415                0.003
Actual Positive       0.573                0.008
ACC: 0.014
Confusion Matrix (Normalized):
                      Predicted Negative                       Predicted Positive
Actual Negative       0.000                0.000
Actual Positive       0.986                0.014


In [32]:
from torch_geometric.nn import ChebConv

class ChebConvModel(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers, dropout=True, K=3):
        """
        ChebConv Model.
        - dropout: Whether to apply dropout.
        - K: Order of Chebyshev polynomials (controls the size of the neighborhood).
        """
        super(ChebConvModel, self).__init__()
        self.dropout = dropout
        self.num_classes = output_dim

        # Define ChebConv layers
        self.layers = torch.nn.ModuleList()
        for i in range(num_layers):
            if i == 0:  # First layer
                self.layers.append(ChebConv(input_dim, hidden_dim, K=K))
            elif i == num_layers - 1:  # Last layer
                self.layers.append(ChebConv(hidden_dim, output_dim, K=K))
            else:  # Middle layers
                self.layers.append(ChebConv(hidden_dim, hidden_dim, K=K))

    def forward(self, x, edge_index):
        h = x
        for i, layer in enumerate(self.layers):
            if self.dropout:
                h = F.dropout(h, p=0.5, training=self.training)
            h = layer(h, edge_index)
            if i < len(self.layers) - 1:  # Apply activation only for hidden layers
                h = F.relu(h)

        if self.num_classes == 1:
            return F.sigmoid(h)
        else:
            return F.log_softmax(h, dim=1)


In [38]:
input_dim = 1
hidden_dim = 32
output_dim = 1
model = ChebConvModel(input_dim, hidden_dim, output_dim, num_layers=4, dropout=True, K=10)


model = train_model(model, train_x, train_y, trainEdges, epochs=500, lr=0.01)


print("based on auc th")

metric,th = test_model(model, test_x, test_y, testEdges-len(train_x), mask = mask_test)
metric,th = test_model(model, test_x, test_y, testEdges-len(train_x), mask = mask_test_sybils, threshold = th)
print("\n\n")
print("based on 0.5")
metric,th = test_model(model, test_x, test_y, testEdges-len(train_x), mask = mask_test, threshold = 0.5)
metric,th = test_model(model, test_x, test_y, testEdges-len(train_x), mask = mask_test_sybils, threshold = 0.5)

  0%|          | 1/500 [00:01<08:47,  1.06s/it]

Epoch 1/500, Loss: 0.1286


  4%|▍         | 20/500 [00:24<10:35,  1.32s/it]

Epoch 20/500, Loss: 0.0293


  8%|▊         | 40/500 [00:44<07:17,  1.05it/s]

...

 96%|█████████▌| 480/500 [08:06<00:21,  1.07s/it]

Epoch 480/500, Loss: 0.0170


100%|██████████| 500/500 [08:26<00:00,  1.01s/it]

Epoch 500/500, Loss: 0.0171
based on auc th


AUC: 0.489
ACC: 0.438
Confusion Matrix (Normalized):
                      Predicted Negative                       Predicted Positive
Actual Negative       0.401                0.017
Actual Positive       0.545                0.037
ACC: 0.063
Confusion Matrix (Normalized):
                      Predicted Negative                       Predicted Positive
Actual Negative       0.000                0.000
Actual Positive       0.937                0.063



based on 0.5
AUC: 0.489
ACC: 0.437
Confusion Matrix (Normalized):
                      Predicted Negative                       Predicted Positive
Actual Negative       0.402                0.016
Actual Positive       0.547                0.035
ACC: 0.060
Confusion Matrix (Normalized):
                      Predicted Negative                       Predicted Positive
Actual Negative       0.000                0.000
Actual Positive       0.940                0.060


#preprocessing based on cliques

In [41]:
G = nx.Graph()
G.add_edges_from(edges)

In [42]:
traing = G.subgraph(trainNodes)
testg = G.subgraph(testNodes)


In [81]:
maximal_cliques = list(nx.find_cliques(traing)) # Bron-Kerbosch algorithm
bclique = 0
sclique = 0
train_x_2 = train_x.clone()
for clique in tqdm(maximal_cliques):
    if len(clique)>3:
        label = 0
        for node in clique:
            label+= train_x_2[node]
        if label<1.5:
            bclique+=1
            for node in clique:
                train_x_2[node] = 0
        elif label>1.5:
            sclique+=1
            for node in clique:
                train_x_2[node] = 1

print(bclique,sclique)

100%|██████████| 40686/40686 [00:00<00:00, 2115663.93it/s]

0 0


In [82]:
maximal_cliques = list(nx.find_cliques(testg)) # Bron-Kerbosch algorithm
bclique = 0
sclique = 0
test_x_2 = test_x.clone()
for clique in tqdm(maximal_cliques):
    if len(clique)>3:
        label = 0
        for node in clique:
            label+= test_x_2[node-37840]
        if label<1.5:
            bclique+=1
            for node in clique:
                test_x_2[node-37840] = 0
        elif label>1.5:
            sclique+=1
            for node in clique:
                test_x_2[node-37840] = 1

print(bclique,sclique)


100%|██████████| 174448/174448 [00:00<00:00, 415555.65it/s]

0 13


In [83]:
input_dim = 1
hidden_dim = 16
output_dim = 1
model = GCN(input_dim, hidden_dim, output_dim, num_layers=3)
model = train_model(model, train_x, train_y, trainEdges, epochs=100, lr=0.01)

print("based on auc th")
metric,th = test_model(model, test_x, test_y, testEdges-len(train_x), mask = mask_test)
metric,th = test_model(model, test_x, test_y, testEdges-len(train_x), mask = mask_test_sybils, threshold = th)

  2%|▏         | 2/100 [00:00<00:08, 11.84it/s]

Epoch 1/100, Loss: 0.0269


 22%|██▏       | 22/100 [00:02<00:05, 13.82it/s]

Epoch 20/100, Loss: 0.0176


 42%|████▏     | 42/100 [00:03<00:03, 16.59it/s]

...

Epoch 60/100, Loss: 0.0170


 82%|████████▏ | 82/100 [00:06<00:01, 16.39it/s]

Epoch 80/100, Loss: 0.0170


100%|██████████| 100/100 [00:07<00:00, 13.92it/s]


Epoch 100/100, Loss: 0.0169
based on auc th
AUC: 0.651
ACC: 0.617
Confusion Matrix (Normalized):
                      Predicted Negative                       Predicted Positive
Actual Negative       0.263                0.155
Actual Positive       0.227                0.354
ACC: 0.609
Confusion Matrix (Normalized):
                      Predicted Negative                       Predicted Positive
Actual Negative       0.000                0.000
Actual Positive       0.391                0.609


In [84]:
input_dim = 1
hidden_dim = 16
output_dim = 1
model = GCN(input_dim, hidden_dim, output_dim, num_layers=3)
model = train_model(model, train_x_2, train_y, trainEdges, epochs=100, lr=0.01)

print("based on auc th")
metric,th = test_model(model, test_x_2, test_y, testEdges-len(train_x), mask = mask_test)
metric,th = test_model(model, test_x_2, test_y, testEdges-len(train_x), mask = mask_test_sybils, threshold = th)

  2%|▏         | 2/100 [00:00<00:06, 15.34it/s]

Epoch 1/100, Loss: 0.0200


 22%|██▏       | 22/100 [00:01<00:04, 16.44it/s]

Epoch 20/100, Loss: 0.0171


 42%|████▏     | 42/100 [00:02<00:03, 17.37it/s]

...

Epoch 60/100, Loss: 0.0169


 82%|████████▏ | 82/100 [00:05<00:01, 12.97it/s]

Epoch 80/100, Loss: 0.0169


100%|██████████| 100/100 [00:06<00:00, 14.84it/s]


Epoch 100/100, Loss: 0.0169
based on auc th
AUC: 0.668
ACC: 0.612
Confusion Matrix (Normalized):
                      Predicted Negative                       Predicted Positive
Actual Negative       0.277                0.141
Actual Positive       0.246                0.335
ACC: 0.576
Confusion Matrix (Normalized):
                      Predicted Negative                       Predicted Positive
Actual Negative       0.000                0.000
Actual Positive       0.424                0.576


In [85]:
input_dim = 1
hidden_dim = 16
output_dim = 1
model = GAT(input_dim, hidden_dim, output_dim, num_layers=4, heads = 4)
model = train_model(model, train_x, train_y, trainEdges, epochs=100, lr=0.01)

print("based on auc th")
metric,th = test_model(model, test_x, test_y, testEdges-len(train_x), mask = mask_test)
metric,th = test_model(model, test_x, test_y, testEdges-len(train_x), mask = mask_test_sybils, threshold = th)

  1%|          | 1/100 [00:00<00:41,  2.40it/s]

Epoch 1/100, Loss: 0.0225


 20%|██        | 20/100 [00:08<00:32,  2.50it/s]

Epoch 20/100, Loss: 0.0178


 40%|████      | 40/100 [00:16<00:24,  2.48it/s]

...

 80%|████████  | 80/100 [00:34<00:07,  2.50it/s]

Epoch 80/100, Loss: 0.0170


100%|██████████| 100/100 [00:42<00:00,  2.33it/s]

Epoch 100/100, Loss: 0.0169
based on auc th


AUC: 0.720
ACC: 0.646
Confusion Matrix (Normalized):
                      Predicted Negative                       Predicted Positive
Actual Negative       0.309                0.109
Actual Positive       0.245                0.337
ACC: 0.579
Confusion Matrix (Normalized):
                      Predicted Negative                       Predicted Positive
Actual Negative       0.000                0.000
Actual Positive       0.421                0.579


In [86]:
input_dim = 1
hidden_dim = 16
output_dim = 1
model = GAT(input_dim, hidden_dim, output_dim, num_layers=4, heads = 4)
model = train_model(model, train_x_2, train_y, trainEdges, epochs=100, lr=0.01)

print("based on auc th")
metric,th = test_model(model, test_x_2, test_y, testEdges-len(train_x), mask = mask_test)
metric,th = test_model(model, test_x_2, test_y, testEdges-len(train_x), mask = mask_test_sybils, threshold = th)

  1%|          | 1/100 [00:00<00:40,  2.47it/s]

Epoch 1/100, Loss: 0.0256


 20%|██        | 20/100 [00:09<00:43,  1.84it/s]

Epoch 20/100, Loss: 0.0177


 40%|████      | 40/100 [00:17<00:24,  2.41it/s]

...

 80%|████████  | 80/100 [00:36<00:10,  1.90it/s]

Epoch 80/100, Loss: 0.0170


100%|██████████| 100/100 [00:44<00:00,  2.24it/s]

Epoch 100/100, Loss: 0.0170
based on auc th


AUC: 0.720
ACC: 0.647
Confusion Matrix (Normalized):
                      Predicted Negative                       Predicted Positive
Actual Negative       0.307                0.112
Actual Positive       0.241                0.340
ACC: 0.585
Confusion Matrix (Normalized):
                      Predicted Negative                       Predicted Positive
Actual Negative       0.000                0.000
Actual Positive       0.415                0.585
